In [1]:
using Revise, LazySets

In [2]:
X = rand(VPolytope, dim=3)
lm = Projection(X, [1,2]);

In [3]:
M = matrix(lm)
M = Matrix(M)

2×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0

In [4]:
an_element(X)

3-element Array{Float64,1}:
  2.915672660849058
 -0.10793685215465766
  1.7668265892518031

In [5]:
X = rand(VPolytope, dim=3);
overapproximate(X, OctDirections)

HPolytope{Float64,SparseArrays.SparseVector{Float64,Int64}}(HalfSpace{Float64,SparseArrays.SparseVector{Float64,Int64}}[HalfSpace{Float64,SparseArrays.SparseVector{Float64,Int64}}(  [1]  =  1.0
  [2]  =  1.0, 1.7487425579928906), HalfSpace{Float64,SparseArrays.SparseVector{Float64,Int64}}(  [1]  =  1.0
  [2]  =  -1.0, 2.083456053755553), HalfSpace{Float64,SparseArrays.SparseVector{Float64,Int64}}(  [1]  =  -1.0
  [2]  =  1.0, 1.8757792504076543), HalfSpace{Float64,SparseArrays.SparseVector{Float64,Int64}}(  [1]  =  -1.0
  [2]  =  -1.0, 1.7109028127304877), HalfSpace{Float64,SparseArrays.SparseVector{Float64,Int64}}(  [1]  =  1.0
  [2]  =  0.0
  [3]  =  1.0, 1.2126633493597014), HalfSpace{Float64,SparseArrays.SparseVector{Float64,Int64}}(  [1]  =  1.0
  [2]  =  0.0
  [3]  =  -1.0, 1.9435388512235137), HalfSpace{Float64,SparseArrays.SparseVector{Float64,Int64}}(  [1]  =  -1.0
  [2]  =  0.0
  [3]  =  1.0, 2.060967155709675), HalfSpace{Float64,SparseArrays.SparseVector{Float64,Int64}}(  [1

In [8]:
using LazySets: projection_matrix

function LazySets.project(V::Union{<:VPolygon{N}, <:VPolytope{N}},
                          block::AbstractVector{Int}) where {N}
    n = dim(V)
    M = projection_matrix(block, n, N)
    πvertices = broadcast(v -> M * v, vertices_list(V))
    return basetype(V)(πvertices)
end

In [10]:
using LazySets.Approximations: AbstractDirections
using LazySets: default_lp_solver

LazySets.overapproximate(X::LazySet{N}, ZT::Type{<:Zonotope},
                         dir::Type{<:AbstractDirections};
                         solver=default_lp_solver(N)) where {N} = overapproximate(X, ZT, dir(dim(X)), solver=solver)

In [16]:
function _overapproximate(P::VPolytope, ::Type{<:Zonotope}, dirs=OctDirections)
    n = dim(P)
    j = 1
    p = Vector{typeof(P)}(undef, Int(floor(n/2)))
    for i=1:2:n-1
        p[j] = project(P, [i, i+1])
        j += 1
    end
   
    if !iseven(n)
        push!(p, project(P, [n]))
    end

    Z = [overapproximate(poly, Zonotope, dirs) for poly in p]
    return reduce(×, Z)
end

_overapproximate (generic function with 2 methods)

In [17]:
n = 10
P = rand(VPolytope, dim=n);

In [19]:
@time _overapproximate(P, Zonotope);
@time overapproximate(P, Zonotope, OctDirections(n));

  0.018742 seconds (16.66 k allocations: 1.045 MiB)
  1.743596 seconds (293.75 k allocations: 10.809 MiB)


---

In [ ]:
V = rand(VPolytope, dim=5)

In [ ]:
project(V, [1, 2, 5])

In [ ]:
V.vertices